In [1]:
# STEP 1: Mount Drive (if not already)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd
from huggingface_hub import InferenceClient
from PIL import Image
import base64
import io
from tqdm import tqdm
import time

# Load 1000-row CSV
df = pd.read_csv("/content/drive/MyDrive/vision_benchmark/metadata/disguises_pairs_sample1000.csv")

# HuggingFace model client
client = InferenceClient(model="meta-llama/Llama-4-Scout-17B-16E-Instruct")

# Encode image to base64
def encode_image(img_path):
    with open(img_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# Collect results
results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        img1 = encode_image(row["img1_path"])
        img2 = encode_image(row["img2_path"])
        prompt = "Do these two people look like the same person, or is one in disguise?"

        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img1}"}},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img2}"}},
                    ]
                }
            ]
        )

        prediction = response.choices[0].message.content

        results.append({
            "img1": row["img1_path"],
            "img2": row["img2_path"],
            "ground_truth": row["label"],
            "prediction": prediction
        })

    except Exception as e:
        results.append({
            "img1": row["img1_path"],
            "img2": row["img2_path"],
            "ground_truth": row["label"],
            "prediction": f"ERROR: {str(e)}"
        })

    time.sleep(1)  # avoid throttling

# Save output
results_df = pd.DataFrame(results)
out_path = "/content/drive/MyDrive/vision_benchmark/metadata/disguise_meta-llama_Llama-4-Scout-17B-16E-Instruct_results.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved results to: {out_path}")


100%|██████████| 1000/1000 [44:12<00:00,  2.65s/it]


✅ Saved results to: /content/drive/MyDrive/vision_benchmark/metadata/disguise_meta-llama_Llama-4-Scout-17B-16E-Instruct_results.csv


In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, f1_score

# Load the CSV
df = pd.read_csv("/content/drive/MyDrive/vision_benchmark/metadata/disguise_meta-llama_Llama-4-Scout-17B-16E-Instruct_results.csv")
results_df.to_csv(out_path, index=False)

# Define broader matching logic for predictions
def flexible_map_prediction(text):
    text = text.strip().lower()
    if any(phrase in text for phrase in [
        "show the same person",
        "appear to be the same person",
        "appear to show the same person"
    ]):
        return 1
    elif any(phrase in text for phrase in [
        "do not appear to be the same person",
        "do not appear to be the same individual"
    ]):
        return 0
    else:
        return -1  # unclear or noisy response

# Apply the mapping
df["flexible_pred"] = df["prediction"].apply(flexible_map_prediction)

# Filter only usable predictions
filtered_df = df[df["flexible_pred"] != -1]
y_true = filtered_df["ground_truth"]
y_pred = filtered_df["flexible_pred"]

# Compute and print metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.4731
Precision: 0.4731
F1 Score: 0.6423
